In [25]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import RandomizedSearchCV

In [26]:
data = pd.read_csv('/Users/sa12/Documents/Repositories/The-CyberChase/DATA/Clean4Model')
NoBwd = pd.read_csv('/Users/sa12/Documents/Repositories/The-CyberChase/DATA/CleanNoBwd')
data = data.drop(columns=['Unnamed: 0'])
NoBwd = NoBwd.drop(columns=['Unnamed: 0'])

In [27]:
# Features and target
X = data.drop(columns=['Country'])
y = data['Country']

In [28]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [29]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [31]:
model = BalancedRandomForestClassifier(n_estimators=100, sampling_strategy = all, random_state=42, bootstrap=False)
model.fit(X_train_scaled, y_train)

/opt/anaconda3/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(
/opt/anaconda3/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:601: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `False` to silence this warning and adopt the future behaviour.
  warn(


AttributeError: 'bool' object has no attribute 'keys'

In [ ]:
y_pred = model.predict(X_test_scaled)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.7518579378429447

Classification Report:
               precision    recall  f1-score   support

           1       0.93      0.91      0.92    106603
           2       0.73      0.49      0.58     66336
           3       0.59      0.68      0.63     38184
           4       0.91      0.84      0.87     34085
           5       0.67      0.73      0.70     17412
           6       0.51      0.65      0.58     15252
           7       0.50      0.90      0.64      9434
           8       0.64      0.76      0.69      7687
           9       0.76      0.91      0.83      7008
          10       0.38      0.67      0.49      6540

    accuracy                           0.75    308541
   macro avg       0.66      0.75      0.69    308541
weighted avg       0.77      0.75      0.75    308541


Confusion Matrix:
 [[97514   645    17     4  1718    26  3627    94    22  2936]
 [ 4155 32324 14438  1681  1400  3796  4495  1297   671  2079]
 [ 1238  1435 25921   759  2644  3889   1

In [ ]:
'''
from scipy.stats import randint

param_distributions = {
    'n_estimators': randint(100, 500),
    'max_depth': [None, 10, 20, 30, 50],
    'max_features': ['sqrt', 'log2', 0.3, 0.5, None],
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 5),
    'bootstrap': [True, False]
}

search = RandomizedSearchCV(
    estimator=model,
    sampling_strategy = all,
    param_distributions=param_distributions,
    replacement = True,
    n_iter=30,
    scoring='f1_macro',  # or 'roc_auc', 'balanced_accuracy'
    cv=5,
    random_state=42,
    verbose=1,
    n_jobs=-1
)

search.fit(X_train, y_train)
print("Best Params:", search.best_params_)
print("Best Score:", search.best_score_)
'''

Fitting 5 folds for each of 30 candidates, totalling 150 fits


/opt/anaconda3/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/opt/anaconda3/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(
/opt/anaconda3/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/

In [32]:
X2 = NoBwd.drop(columns=['Country'])
y2 = NoBwd['Country']
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=42)
X_train2_scaled = scaler.fit_transform(X_train2)
X_test2_scaled = scaler.transform(X_test2)


In [33]:
model2 = BalancedRandomForestClassifier(n_estimators=100, sampling_strategy = all, bootstrap=False, random_state=42)
model2.fit(X_train2_scaled, y_train2)
y_pred2 = model.predict(X_test2_scaled)
print("Accuracy:", accuracy_score(y_test2, y_pred2))
print("\nClassification Report:\n", classification_report(y_test2, y_pred2))
print("\nConfusion Matrix:\n", confusion_matrix(y_test2, y_pred2))

/opt/anaconda3/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(


AttributeError: 'bool' object has no attribute 'keys'